#Day67_DBX_Spark_Egress[write] - json
## Serialized Files - Datalake
### ORC, Parquet, delta (hive)

In [0]:
# extract
ingest_df1=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header",header=True,inferSchema=True)
display(ingest_df1.limit(5))

#major write formats 
##structured (csv)
##semi-structured (json/xml)
##datalake (serialized files(parquet,orc,delta))
##lakehouse (table(delta/hive tables))


# Serialization is a Data Mechanics
- mostly used in Bigdata
- Encoding format performant for savin space + processing inteligent bigdata format
- Binary format (Brainy file format) / byte code
  - orc, parquet, **delta (databricks properatory)**  => columnar formats
  - orc optimized row column fo
  - parquest tiled data format
  - delta enriched parquet; delta (change/modification) operations can be performed(DML **ACID property**)
- Benefits: fast, compact, interoperable, extensible, scalable , secured
  - FAst - brings only required
  - Compact - stores data more compactly
  - Interoperable - works in various systems
- by default orc/parquet uses snappy compression
  - compression='zlib', 'lzo', 'gzip'.. in write operation  
- Deserilize - uncompression

## other Data Mechanics
- Encoding / Decoding
- Compress / Decompress: eg snappy - good compressor in bigdata
- Encrypt / Decrypt
- Masking / redaction (encoding without decoding - non reversable)

### delta -> developed on top of parquest with advancedc features by databricks
- with some additional files we can in the storage
  - log info, what change happend, incremental changes,.. 

- Now its not directly supported by spark like orc and parquest; we have to use format('delta') option
  - df.wrtie.orc()
  - df.wrtie.parquet()
  - df.wrtie.format('delta') ; other serialized not directly supported by spark are avero, iceberg .. 

In [0]:
# orc
ingest_df1.write.orc("/Volumes/workspace/wd36schema/ingestion_volume/target/orcout",mode='overwrite')
# parquet
ingest_df1.write.parquet("/Volumes/workspace/wd36schema/ingestion_volume/target/parquetout",mode='overwrite')

In [0]:
spark.read.orc("/Volumes/workspace/wd36schema/ingestion_volume/target/orcout").where("profession='Pilot'").display()
## orc will bring only required data unlike csv it will fetch nall data then apply filter

In [0]:
# explain the plan
spark.read.orc("/Volumes/workspace/wd36schema/ingestion_volume/target/orcout").where("profession='Pilot'").explain()

In [0]:
#spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/target/csvout",sep='~').show()
spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/target/csvout",sep='~').where("_c3='Pilot'").explain()